# 1. Validation des données restructurées par rapport à un schéma à l'aide d'un tableur

<div class="alert alert-block alert-warning">
    <b>A la fin de la formation, vous pourrez:</b>
    <br>
    <ul>
        <li>Comprendre et utiliser des définitions standard pour écrire un schéma JSON pour la validation des données.</li>
        <li>Valider des données à l'aide de Microsoft Excel.</li>
        <li>Apprendre à valider sur des schémas définis les données exploitables par machine en utilisant des applications en ligne.</li>
    </ul>
</div>

---

## 1.1 Création d'un schéma JSON

Lorsque vous avez produit votre fichier exploitable par machine dans la leçon 1, vous avez imaginé votre propre approche pour structurer la ligne d'en-tête et les données. Vous avez nommé les colonnes vous-même, et décidé du nombre de colonnes et des données qu'elles devaient contenir. Pour ce faire, vous avez passé en revue les données sources.

Cependant, cette approche pose des problèmes :

- Comment allez-vous maintenir la cohérence entre les mises à jour ? 
- Comment différentes personnes, travaillant sur des données similaires ou des mises à jour consécutives, assureront-elles la cohérence entre elles ?
- Comment saurez-vous si le fichier restructuré que vous avez créé est réellement lisible par la machine ?

Si un programmeur voulait construire une application - disons une application qui aide les gens à trouver l'arrêt de bus le plus proche - et que vous modifiez la structure des mises à jour d'une version à l'autre, il ne pourra pas automatiser son programme, et cela pourrait l'empêcher de le construire.

La première étape du cycle de vie des données n'est pas simplement d'entrer dans vos données et de les restructurer. Vous devez d'abord définir un `schéma`.

![Cycle de vie des données](images/data-lifecycle-fr.jpg)

Votre `schéma` définit les métadonnées structurelles pour votre processus de préparation. Ce n'est pas le format dans lequel vos données d'entrée arrivent, mais c'est ce à quoi vous voulez qu'elles ressemblent lorsque vous avez terminé.

Les termes standard utilisés sont définis par [JSON Schema](https://json-schema.org/learn/) et affinés pour être utilisés dans la structuration des données CSV dans le [Table Schema](https://specs.frictionlessdata.io/table-schema/) de Frictionless Data.

En termes simples, vous devez spécifier les colonnes dans lesquelles un CSV ou un fichier Excel d'entrée sera restructuré. Les nouvelles colonnes sont définies par les champs de votre schéma. Ces champs cibles seront probablement ceux de votre base de données ou de votre logiciel d'analyse. Tant que vos données d'entrée ne seront pas conformes à cette structure, vos données ne seront pas validées.

### 1.1.1 Exigences minimales valides

Un schéma minimum valide nécessite un `name` pour identifier le schéma, et un seul `field`  minimum valide contenant un `name` et un `type` :

    {
        "name": "Un nom simple",
        "fields": [
            {
                "name":  "Nom du champ, par exemple 'nom_colonne'",
                "type": "Type de données valide, par exemple 'string', 'number'"
                }
            }
        ]
    }

Tout le reste est facultatif, sauf si ce type de champ l'exige spécifiquement.

Un schéma JSON ressemble à un dictionnaire Python, mais faites très attention aux `,` en fin de lignes. Le dernier élément d'une liste ne doit PAS comporter de virgule. 

> Vous pouvez considérer un schéma `field` comme une `column` dans une table, ou un `field` dans une base de données. Les champs ont un `type`, tel qu'un entier ou du texte.

La valeur des `fields` est une liste, ou - dans la terminologie JSON - un `array` d'`objects` du dictionnaire. Il n'est pas surprenant que chaque champ ait un `name`, un `title` et une `description`, dont seul le `name` est requis. 

### 1.1.2 Types

Les champs ont également un `type`. Celui-ci décrit les données attendues et limite les actions qui peuvent être effectuées pendant le processus de préparation des données :

* `string`: toute chaîne de texte.
* `number`: toute valeur numérique, y compris les nombres entiers et les nombres flottants.
* `integer`: toute valeur basée sur des nombres entiers.
* `boolean`: une valeur booléenne [`true`, `false`].
* `object`: toute donnée JSON valide.
* `array`: toute donnée valide basée sur un array.
* `date`: toute date sans heure. Doit être au format ISO8601, `YYYY-MM-DD`.
* `datetime`: toute date avec une heure. Doit être au format ISO8601, avec l'heure UTC spécifiée (en option) comme `YYYY-MM-DD hh:mm:ss Zz`.
* `year`: toute année, formatée comme `YYYY`.

Il existe d'autres [types et formats](https://specs.frictionlessdata.io/table-schema/#types-and-formats) comme les `geojson`, les `geopoints` et les variations de dates.

### 1.1.3 Contraintes

En outre, ces données peuvent avoir des contraintes (`constrained`):

* `required`: booléen, indique si ce champ est obligatoire (mais les valeurs en blanc dans la colonne de saisie sont autorisées et seront mises à la valeur par défaut `missingValues`)
* `unique`: booléen, si `true` alors toutes les valeurs de cette colonne d'entrée doivent être uniques
* `minimum`: `integer` / `number`, selon le cas, définissant le nombre minimum de caractères dans une chaîne, ou les valeurs minimum des nombres ou des entiers
* `maximum`: `integer` / `number`, selon le cas, définissant le nombre minimum de caractères dans une chaîne, ou les valeurs minimum des nombres ou des entiers
* `enum`: une liste (ou un array) de termes uniques autorisés dans ce champ

A titre d'exemple :

    {
        "name": "Un nom simple",
        "fields": [
            {
                "name": "Nom du champ, par exemple 'nom_colonne'",
                "type": "Type de données valide, par exemple 'string', 'number'",
                "constraints": {
                    "enum": ["cheddar", "gouda", "autre"]
                }
            }
        ]
    }

Là encore, il existe d'autres [constraints](https://specs.frictionlessdata.io/table-schema/#constraints), telles que `pattern`, `maxLength`, `minLength` que vous pouvez également utiliser.

### 1.1.4 Autres propriétés

Il existe également des propriétés spéciales que vous pouvez ajouter à votre schéma et qui ne font pas partie des définitions des `fields`:

* `missingValues`: définit quels termes de vos données doivent être traités comme des valeurs manquantes, par exemple `-`, `NaN`, `..`, etc. Il doit être présenté sous forme de liste, avec les termes définis comme des chaînes de caractères, par exemple `["NaN", ".."]`

### 1.1.5 Exemple de schéma

A titre d'exemple, imaginons que nous voulons que nos données de destination soient conformes à la structure suivante :

    =========  ============  =============  ========  ================  =====================  =============  ========================
      la_code        ba_ref  occupant_name  postcode  occupation_state  occupation_state_date  prop_ba_rates  occupation_state_reliefs
    =========  ============  =============  ========  ================  =====================  =============  ========================
    E06000044  177500080710  A company       PO5 2SE              True             2019-04-01          98530  [small_business, retail]
    =========  ============  =============  ========  ================  =====================  =============  ========================

Le schéma complet de cet exemple est donc le suivant :

In [ ]:
{
    "name": "test_schema",
    "title": "Test Schema",
    "description": "A test Schema",
    "fields": [
        {
            "name": "la_code",
            "title": "Local authority code",
            "type": "string",
            "description": "Standard code for local authority."
        },
        {
            "name": "ba_ref",
            "title": "Billing reference",
            "type": "string",
            "description": "Unique code for a specific hereditament. May be multiple rows for history."
        },
        {
            "name": "prop_ba_rates",
            "title": "Property billing rates",
            "type": "number",
            "description": "Actual rates paid by a specific ratepayer."
        },
        {
            "name": "occupant_name",
            "title": "Occupier name",
            "type": "string",
            "description": "Name of the ratepayer."
        },
        {
            "name": "postcode",
            "title": "Postcode",
            "type": "string",
            "description": "Full address or postcode of ratepayer."
        },
        {
            "name": "occupation_state",
            "title": "Occupation state",
            "type": "boolean",
            "description": "Occupation status, void or occupied."
        },
        {
            "name": "occupation_state_date",
            "title": "Date of occupation state",
            "type": "date",
            "description": "Date of the start of status in occupation_state."
        },
        {
            "name": "occupation_state_reliefs",
            "title": "Occupation state reliefs",
            "type": "array",
            "description": "Array of the categories of reliefs / exemptions applied.",
            "constraints": {
                "enum": [
                    "small_business", 
                    "rural", 
                    "charity", 
                    "enterprise_zone", 
                    "vacancy", 
                    "hardship",
                    "retail", 
                    "discretionary", 
                    "exempt", 
                    "transitional", 
                    "other"
                ]
            }
        }
    ],
    "missingValues": ["NaN", ".."]
}

__Sauvegarder__ votre schéma JSON dans un éditeur de texte ordinaire (__PAS__ Word ou tout ce qui peut y ajouter du XML) sous la forme d'un fichier `.json` (par exemple `my_schema.json`).

Un très bon éditeur de texte est [Notepad++](https://notepad-plus-plus.org/downloads/). Utilisez-le pour créer et enregistrer votre JSON.

Maintenant, à vous de jouer :

<div class="alert alert-block alert-success">
    <p><b>Exercice:</b></p>
    <p>En utilisant le fichier restructuré que vous avez créé à la leçon 1.1, créez un schéma JSON. N'oubliez pas que chaque <code>field</code> <code>name</code> doit être identique au nom de l'en-tête de la colonne. Définissez le champ <code>type</code> et - si vous les avez - toutes les <code>missingValues</code>.</p>
</div>

Dans la section suivante, vous apprendrez comment spécifier les types de valeurs dans une feuille de calcul.

<div class="alert alert-block alert-info">
    <b>Références:</b>
    <br>
    <ul>
        <li><a href="https://json-schema.org/learn/">JSON Schema</a></li>
        <li><a href="https://specs.frictionlessdata.io/table-schema/">Frictionless Data Table Schema</a></li>
        <li><a href="https://notepad-plus-plus.org/downloads/">Notepad++</a></li>
    </ul>
</div>

---

## 1.2 Appliquer des processus de validation de données à des cellules dans un tableur

Vos  `types` - à ce stade - ne sont qu'un guide. Vous n'aurez pas de retour d'information, ni de messages d'erreur comme ceux que vous obtenez lorsque vous exécutez du code Python, si l'un des types de données de vos colonnes de champ est erroné. Il y a plusieurs façons d'obtenir ce retour d'information pour que vous puissiez corriger les choses, mais nous commencerons par la validation des données dans les cellules de la feuille de calcul.

Ce qui suit est adapté d'un [tutoriel Microsoft Office](https://support.office.com/en-gb/article/apply-data-validation-to-cells-29fecbcc-d1b9-42c1-9d76-eff3ce5f7249). Cette approche fonctionnera aussi bien dans OpenOffice que dans Google Sheets, bien que les étapes spécifiques soient différentes.

Microsoft a un fichier d'exemple que vous pouvez [télécharger](http://download.microsoft.com/download/9/6/8/968A9140-2E13-4FDC-B62C-C1D98D2B0FE6/Data%20Validation%20Examples.xlsx).

### 1.2.1 Spécifier la validation pour les types de données

NB: dans ce paragraphe nous vous indiquons les instructions sous excel à la fois en français dans le texte et en anglais en image. Vous devriez pouvoir trouver les options si vous avez une version française ou anglaise.

Le processus est simple :

1. Sélectionnez les cellules d'une colonne spécifique que vous souhaitez limiter par type
2. Sélectionnez __Données > Outils de données > Validation des données__.

  ![Validation des données Excel](images/excel-data-validation.png)

3. Dans l'onglet __Option__, sous __Autoriser__, sélectionnez l'un des éléments suivants:

  ![Paramètres de validation](images/excel-validation-settings.jpg)
 
  - __Nombre Entier__: restreindre la cellule pour n'accepter que des valeurs `integer`.
  - __Décimal__: restreint la cellule pour n'accepter que les valeurs `float` ou `number`.
  - __Liste__: choisir des données dans une liste déroulante, et limitée par les valeurs contraintes par `enum`.
  - __Date__: restreindre la cellule pour n'accepter que les `date`.
  - __Heure__: restreindre la cellule pour n'accepter que les `datetime`.
  - __Longueur de Texte__: restreindre la longueur du texte, équivalent aux contraintes `maxLength`.
  - __Personalisé__: pour une formule personnalisée.
 
4. Sous __Données__, vous pouvez sélectionner une condition :

  - Comprise entre
  - non comprise entre
  - égale à
  - différente de
  - supérieure à
  - inférieure à
  - supérieure ou égale à
  - inférieure ou égale à

5. Définissez les autres valeurs requises, en fonction de ce que vous avez choisi pour __Autoriser__ et __Données__. Par exemple, si vous faites un choix entre les valeurs __Minimum:__ et __Maximum:__ pour la ou les cellules.
6. Cochez la case __Ignorer si vide__ si vous souhaitez ignorer les espaces vides (c'est-à-dire pour les données manquantes). Notez, cependant, qu'Excel n'a pas de place pour les caractères spéciaux que vous pourriez utiliser comme marqueur pour les données manquantes (par exemple `..`), de sorte que ceux-ci seront mentionnés comme des erreurs.
7. Sélectionnez __OK__.

Maintenant - seulement pour les nouvelles données - si un utilisateur essaie d'entrer une valeur qui n'est pas valide, une fenêtre pop-up apparaît avec le message "Cette valeur ne correspond pas aux restrictions de validation des données pour cette cellule". Nous allons bientôt lancer la validation de vos données existantes, mais d'abord un détour par les listes (`lists`).

### 1.2.2 Les listes sont un type spécial

Avant de pouvoir valider un type  `list`, vous devez spécifier des termes valables. Dans Excel, cela nécessite une [série d'étapes supplémentaires](https://support.office.com/en-us/article/create-a-drop-down-list-7693307a-59ef-400a-b769-c5402dce407b).

1. Créez une nouvelle feuille de calcul dans Excel, et mettez-y la liste des termes que vous voulez définir comme valeurs valides. Vous pouvez rapidement convertir votre liste en tableau en sélectionnant n'importe quelle cellule de la plage, et en appuyant sur __Ctrl+L__ (attention ces raccourcis pouvent variés d'une versoin d'excel à une autre).

  ![Liste de termes](images/excel-list-terms.png)
  
2. Ajoutez les données de votre liste et formatez-les sous forme de table (__Accueil > Styles > Mettre sous forme de tableau__).
3. Vous pouvez nommer votre table à partir de l'onglet outils de la table - celle-ci pourrait s'appeler "CityTable".  Cela vous aidera à garder une trace de plusieurs tables.
4. Dans le processus de validation indiqué ci-dessus, allez à 3. et sélectionnez __Liste__, puis ajoutez une plage nommée ou un nom de table pour votre liste. 
5. Indiquez une source pour vos termes via l'onglet __Données > Validation de Données > Autoriser > Liste > Source__. Ensuite, spécifiez votre liste de termes comme étant n'importe laquelle :

  ![Source de la liste](images/excel-list-source.png)

  - Vous pouvez sélectionner directement la feuille de liste et la plage (par exemple `=Sheet1!A4:A10`)
  - Convertissez votre liste en une table avec __Ctrl+L__, puis à partir de l'onglet __Création de Tableau__ donnez un nom à votre table, ce qui vous permet de faire référence au nom de la table et à la colonne (par exemple `=CityTable[City]`)
  - Dans l'onglet __Formules__, sélectionnez __Gestionnaire de Nom__, créez un __Nouveau__ élément avec un nom approprié (par exemple `CityList`), et référencez les cellules (par exemple `=Sheet1!A4:A10`), ce qui vous permet ensuite de référencer votre liste n'importe où (par exemple `=CityList`)

### 1.2.3 Valider et obtenir des messages d'erreur pour vos données existantes

Après avoir précisé les règles de validation de vos données existantes, vous risquez d'être déçu. Excel ne vous informe pas automatiquement si ces cellules contiennent des données non valides. Voici un moyen rapide de [mettre en évidence les cellules non valides existantes](https://support.office.com/en-us/article/more-on-data-validation-f38dee73-9900-4ca6-9301-8a5f6e1f0c4c) en encerclant les valeurs:

  ![Encerclez les données non valides](images/excel-circled-cell.gif)

1. Pour appliquer les cercles, sélectionnez les cellules que vous voulez évaluer et allez à __Données > Outils de données > Validation de Données > Entourer les Données non valides__.

  ![Liste des termes](images/excel-data-circle.png)
  
2. Si vous corrigez une entrée non valable, le cercle disparaît automatiquement.
3. Pour supprimer la validation des données pour une cellule, sélectionnez-la, puis allez à __Données > Outils de données > Validation de Données > Options > Effacer Tout__.

C'est maintenant à votre tour :

<div class="alert alert-block alert-success">
    <p><b>Exercice:</b></p>
    <p>En utilisant le fichier restructuré que vous avez créé dans la leçon 1.1, spécifiez les critères de validation pour chaque colonne en vous basant sur le schéma JSON que vous avez créé. Vérifiez les données non valides et corrigez-les si nécessaire.</p>
    <p> Une chose que vous pourriez remarquer dans vos données ... parfois vous avez un entier non valide spécifié comme une plage, par exemple <code>200-210</code>. Voici quelques idées sur la façon de traiter ce problème.
    <ul>
        <li><b>Fourchettes au lieu de nombres</b> : si, par exemple, votre fourchette est <code>200-210</code>, vous pourriez réinitialiser cette valeur en tant que <code>200</code>, ou <code>210</code> ou même la moyenne de la fourchette <code>205</code>. Quoi que vous décidiez, documentez votre décision dans votre fichier de métadonnées .</li>
        <li><b>Fourchettes de dates</b> : il en va de même pour les dates, bien que vous deviez être prudent ... il est probable qu'une valeur s'applique à la fin de la fourchette de dates, et non au début ou au milieu, donc par exemple <code>2008-2009</code> est le plus susceptible d'être <code>2009</code></li>
    </ul>
</div>

<div class="alert alert-block alert-info">
    <b>Références:</b>
    <br>
    <ul>
        <li><a href="https://support.office.com/en-gb/article/apply-data-validation-to-cells-29fecbcc-d1b9-42c1-9d76-eff3ce5f7249">Apply data validation to cells</a></li>
        <li><a href="https://support.office.com/en-us/article/create-a-drop-down-list-7693307a-59ef-400a-b769-c5402dce407b">Create a drop-down list</a></li>
        <li><a href="https://support.office.com/en-us/article/more-on-data-validation-f38dee73-9900-4ca6-9301-8a5f6e1f0c4c">More on data validation</a></li>
    </ul>
</div>

---

## 1.3 Enregistrement de votre fichier validé en tant que valeur séparée par des virgules

Les fichiers de valeurs séparées par des virgules (`.csv`) sont des fichiers texte dans lesquels le caractère de virgule `,` sépare chaque champ de texte. Lorsqu'une virgule apparaît dans la valeur - qu'il s'agisse d'une `string` ou `number` - la valeur est alors entourée de guillemets, par exemple `100, 200, "20,000"` indique trois valeurs dans trois champs séparés.

Vous pouvez modifier le caractère de séparation utilisé dans les fichiers texte délimités et .csv, et il existe un large éventail de formats (par exemple `;`, `*`). Il y a un grand nombre de raisons à cela, et c'est en partie pour cette raison que les fichiers au format CSV ne sont pas la panacée que nous pourrions espérer pour assurer la cohérence des données ouvertes.

Dans Excel, vous pouvez [exporter une feuille de calcul](https://support.office.com/en-gb/article/import-or-export-text-txt-or-csv-files-5250ac4c-663c-47ce-937b-339e391393ba) au format CSV en utilisant __Save As__ comme suit :

1. Allez à __Fichier > Enregistrer une copie__.
2. Cliquez sur __Parcourir__.
3. Dans la boîte de dialogue __Enregistrer sous__, dans la case __Type__, choisissez le format de fichier texte pour la feuille de travail ; par exemple, cliquez sur __CSV UTF-8 (délimité par des virgules)__ ou __CSV (séparateur: point virgule)__ (les options peuvent variées en fonction de la version d'Excel).
4. Naviguez jusqu'à l'emplacement où vous souhaitez enregistrer le nouveau fichier texte, puis cliquez sur __Enregistrer__.

> Vous ne pouvez exporter que la feuille de calcul actuelle (c'est-à-dire celle qui est en vue lorsque vous terminez ce processus) vers le nouveau fichier CSV. Vous pouvez enregistrer d'autres feuilles de calcul en tant que fichiers texte séparés en répétant cette procédure pour chaque feuille de calcul.

> Toutes les caractéristiques spécifiques de la feuille de calcul seront perdues. Le formatage (gras, couleurs, etc.), les formules et les critères de validation seront supprimés, ne laissant que les données dans un fichier texte.

---

## 1.4 Valider vos données et du schéma JSON avec CSVLint

Validation de vos données et du schéma JSON à l'aide de CSVLint

Dans la prochaine leçon, nous apprendrons comment valider vos données en utilisant Python directement dans un cahier Jupyter. Pour l'instant, nous utiliserons une ressource en ligne fournie par l'Open Data Institute appelée [CSVLint](https://csvlint.io/).

CSVLint fournit le guide suivant sur la façon de l'utiliser :

![CSVLint](images/csv-lint.jpg)

1. Indiquez l'emplacement du fichier CSV que vous souhaitez vérifier, ou téléchargez-le s'il n'est pas déjà sur le web.
2. Cochez __Add optional schema (in .json format)__ et entrez son emplacement ou téléchargez-le.
3. Appuyez sur le gros bouton __Validate__.

Vous verrez une page qui vous indique comment améliorer votre fichier CSV, s'il a besoin d'être amélioré.

- __Errors__ ont des choses que vous devez vraiment corriger, car elles empêcheront les gens de pouvoir utiliser vos données
- __Warnings__ sont des choses que vous devriez corriger si vous le pouvez, car cela aidera les gens à utiliser vos données
- __Information__ sont des conseils et des astuces qui peuvent faciliter l'utilisation des données

CSVLint énumère également un certain nombre d'erreurs courantes auxquelles vous devez faire attention :

- __Invalid encoding__: si un fichier contient des caractères impairs qui pourraient causer des erreurs d'encodage
- __Line breaks__: si les sauts de ligne ne sont pas les mêmes dans tout le fichier
- __Undeclared header__: si vous ne spécifiez pas de manière lisible par la machine si votre CSV comporte ou non une ligne d'en-tête
- __Ragged rows__: si chaque ligne du fichier n'a pas le même nombre de colonnes
- __Blank rows__: s'il y a des rangées vides
- __Stray/Unclosed quote__: s'il y a des citations non fermées dans le dossier
- __Whitespace__: s'il y a des espaces entre les virgules et les guillemets autour des champs

<div class="alert alert-block alert-info">
    <b>Références:</b>
    <br>
    <ul>
        <li><a href="https://csvlint.io/">CSVLint</a></li>
        <li><a href="https://datagraft.io/">DataGraft</a></li>
    </ul>
</div>

---

## 1.5 Tutoriel de la leçon

<div class="alert alert-block alert-success">
    <p><b>Tutoriel:</b></p>
    <p>Terminez le traitement du fichier avec lequel vous avez commencé à travailler dans la leçon 1.</p>
    <ul>
        <li><b>JSON schéma</b> : Définissez un schéma JSON pour décrire et préciser les critères de validation de votre ensemble de données.
        <li><b>Vérifier les données non valides</b> : Spécifier les critères de validation dans votre application tableur, vérifier les données non valides et les corriger si nécessaire..</li>
        <li><b>Save as CSV</b> : Exportez vos données exploitables par machine depuis votre tableur et enregistrez-les sous forme de <code>.csv</code>.</li>
        <li><b>Valider</b> : Téléchargez votre schéma CSV et JSON sur <a href="https://csvlint.io/">CSVLint</a> et assurez-vous que vous êtes validé avec succès. Corrigez les erreurs que vous recevez.</li>
    </ul>
</div>

Veuillez compléter le tutoriel avant de poursuivre cette série.